In [2]:
! pip install pyspark
! pip install findspark

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 316.9 MB 7.7 kB/s  eta 0:00:01
     |████████████████████████████████| 200 kB 71.3 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425367 sha256=a4aa2df9b3afc1ee34fabcbe56e0c038ec60a4e3bba4688363ebcb046ee2c2c0
  Stored in directory: /home/ubuntu/.cache/pip/wheels/a6/ce/f9/17d82c92f044018df2fe30af63ac043447720d5b2cee39b40f
Successfully built pyspark


In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .config('spark.executor.cores', '2')
        .config('spark.executor.instances', '4')
        .config("spark.executor.memory", "3g")
        .config("spark.driver.memory", "2g")
        .getOrCreate()
)

sql = SQLContext(spark)

### Split files

In [116]:
# name_files = [x.split(' ')[-1].split('/')[-1] for x in files.split('\n')]
# name_files.remove('train.csv')

['2019-08-22.txt', '2019-09-21.txt', '2019-10-21.txt']

In [ ]:
import pickle

with open("name_files.pickle", "rb") as output_file:
    name_files = pickle.load(output_file)

# Вариант чтения 1 

In [4]:
defaultFS = spark._jsc.hadoopConfiguration().get("fs.defaultFS")
defaultFS

'hdfs://rc1a-dataproc-m-skgoxg37ozdzyuje.mdb.yandexcloud.net'

In [13]:
table = spark.read.format('csv').options(header='true', inferSchema='true').load(defaultFS + '/user/hive/warehouse/2019-08-22.txt')

In [11]:
table.columns

['# tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario']

# Вариант чтения 2

In [95]:
import pyspark.sql.functions as f

def read_txt_files(name):
    data = spark.read.text("s3a://hw2-mlops-course/" + name)

    name_cols = ['tranaction_id',
                 'tx_datetime',
                 'customer_id',
                 'terminal_id',
                 'tx_amount',
                 'tx_time_seconds',
                 'tx_time_days',
                 'tx_fraud',
                 'tx_fraud_scenario']

    data = data.withColumn('temp', f.split('value', ','))\
                .select(*(f.col('temp').getItem(i).alias(name_col) for i, name_col in enumerate(name_cols)))\
                .filter(f.col('tx_datetime').isNotNull())
    
    return data

In [ ]:
for name_file in name_files:
    df = read_txt_files(name_file).limit(10000)
    
    clear_name = name_file.split('.')[0]
    
    #df.write.mode('overwrite').parquet('/user/hive/warehouse/' + clear_name + '.parquet')

# Вариант чтения 3

In [ ]:
from pyspark.sql.types import *

struct = StructType([
    StructField("tranaction_id", IntegerType(), nullable = True),
    StructField("tx_datetime", StringType(), nullable = True), 
    StructField("customer_id", IntegerType(), nullable = True), 
    StructField("terminal_id", IntegerType(), nullable = True),
    StructField("tx_amount", DoubleType(), nullable = True), 
    StructField("tx_time_seconds", IntegerType(), nullable = True), 
    StructField("tx_time_days", IntegerType(), nullable = True),
    StructField("tx_fraud", IntegerType(), nullable = True), 
    StructField("tx_fraud_scenario", IntegerType(), nullable = True)])

In [ ]:
def read_df(name):
    df = spark.read\
          .option("header", "true")\
          .option("inferSchema", "false")\
          .option("delimiter", ",")\
          .schema(struct)\
          .csv("s3a://hw2-mlops-course/" + name)
    
    return df